# Ćwiczenie 1
## # Sprawozdanie: Klasyfikacja za pomocą pojedynczego neuronu uczonego metodą bezgradientową

## Cel ćwiczenia
Celem zadania było zaprojektowanie modelu neuronu (bazującego na modelu McCullochaPittsa) z dwoma wejściami, który potrafi klasyfikować punkty należące do dwóch liniowo
separowalnych zbiorów. Po zakończeniu procesu uczenia sprawdzono, z jaką skutecznością neuron
poprawnie klasyfikuje dostarczone punkty.

## Wstęp teoretyczny
Pojedynczy neuron stanowi podstawowy element sieci neuronowych, a jego działanie opiera się na matematycznym modelu zaproponowanym przez McCullocha i Pittsa. Model ten opisuje neuron jako funkcję przyjmującą zestaw wejść, które są ważone i sumowane, a następnie przepuszczane przez funkcję aktywacji.

W tym ćwiczeniu zastosowano funkcję Heaviside’a, która określa próg aktywacji.

$$
f(x) =
\begin{cases}
1, & \text{dla } x > 0 \\
0, & \text{dla } x \leq 0
\end{cases}
$$


Neuron może być wykorzystany do klasyfikacji punktów w przestrzeni dwuwymiarowej, o ile są one liniowo separowalne. Klasyfikacja ta polega na znalezieniu odpowiednich wag \( w_1, w_2 \) oraz przesunięcia \( b \), które definiują prostą decyzyjną:

$$
    y = f(w_1 \cdot x_1 + w_2 \cdot x_2 + b)
$$

Uczenie metodą bezgradientową polega na iteracyjnej korekcie wag i przesunięcia w taki sposób, aby dla każdego punktu błędy klasyfikacji były eliminowane. Proces ten kontynuowany jest aż do momentu uzyskania poprawnej klasyfikacji całego zbioru uczącego.

## Implementacja w języku Python

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns


## 1. Definicja funkcji aktywacji

### Rola funkcji aktywacji - Heaviside’a
Funkcja decyduje o tym, czy neuron zostanie aktywowany (czyli czy zwróci wartość 1) lub pozostanie nieaktywny (zwróci 0). W kontekście perceptronu funkcja ta odpowiada za podział przestrzeni wejściowej na dwie klasy, co pozwala na liniową separację zbioru danych. Wartość wyjściowa neuronu zależy od sumy ważonej wejść i przesunięcia (biasu). Jeśli suma ta przekracza 0, neuron aktywuje się, w przeciwnym razie pozostaje nieaktywny.

Dzięki takiemu podejściu perceptron może klasyfikować punkty na podstawie prostej decyzyjnej. Jest to szczególnie przydatne w problemach, w których klasy można oddzielić za pomocą jednej linii.

# Funkcja Heaviside'a

In [ ]:
def heaviside(x):
    return np.where(x > 0, 1, 0)

## 2. Implementacja perceptronu
Perceptron przechowuje wagi, bias oraz umożliwia uczenie się na podstawie dostarczonych danych.

In [ ]:
class Perceptron:
    def __init__(self, learning_rate=0.1, n_iter=10):
        self.learning_rate = learning_rate
        self.n_iter = n_iter

    def fit(self, X, y):
        # Inicjalizacja wag (pierwszy element to bias)
        self.weights = np.zeros(1 + X.shape[1])

        # Pętla po iteracjach
        for _ in range(self.n_iter):
            for xi, target in zip(X, y):
                # Obliczenie różnicy między wartością oczekiwaną a przewidywaną
                update = self.learning_rate * (target - self.predict_single(xi))
                # Aktualizacja wag (wraz z biasem)
                self.weights[1:] += update * xi
                self.weights[0] += update
        return self

    def net_input(self, X):
        # Obliczenie sumy ważonej (wraz z biasem)
        return np.dot(X, self.weights[1:]) + self.weights[0]

    def predict_single(self, x):
        # Funkcja aktywacji: zwraca 1, jeśli net_input >= 0, w przeciwnym razie 0
        return np.where(self.net_input(x) >= 0.0, 1, 0)

    def predict(self, X):
        # Predykcja dla wielu punktów
        return np.where(self.net_input(X) >= 0.0, 1, 0)


## Dane treningowe

Tworzymy zbiór punktów, które są liniowo separowalne. Każdy punkt jest oznaczony jako należący do jednej z dwóch klas.

In [ ]:
np.random.seed(42)  # Ustalenie ziarna dla powtarzalności wyników

# Tworzymy dwa zbiory punktów:
# Klasa 0: punkty skupione wokół (-1, -1)
X0 = np.random.randn(50, 2) - [1, 1]
# Klasa 1: punkty skupione wokół (1, 1)
X1 = np.random.randn(50, 2) + [1, 1]

# Łączymy dane treningowe
X_train = np.vstack((X0, X1))
y_train = np.array([0] * 50 + [1] * 50)


## 4. Trenowanie perceptronu
Tworzymy instancję perceptronu i trenujemy go na wcześniej przygotowanych danych.

In [ ]:
perceptron = Perceptron(learning_rate=0.1, n_iter=10)
perceptron.fit(X_train, y_train)

# Predykcje dla danych treningowych (opcjonalnie można sprawdzić, czy klasyfikacja jest poprawna)
train_predictions = perceptron.predict(X_train)
print("Predykcje na zbiorze treningowym:", train_predictions)

## 5. Testowanie perceptronu
Po zakończeniu procesu uczenia sprawdzamy działanie perceptronu na nowych punktach.

In [ ]:
# Definiujemy kilka punktów testowych
new_points = np.array([
    [-2, -2],
    [2, 2],
    [0, 0],
    [-1.5, -0.5],
    [1.5, 0.5]
])

test_predictions = perceptron.predict(new_points)
print("Predykcje dla nowych punktów:", test_predictions)

## 6. Wizualizacja wyników
Przedstawiamy wyniki klasyfikacji oraz granicę decyzyjną perceptronu na wykresie.

In [ ]:
# Ustalamy zakres wykresu
x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                     np.arange(y_min, y_max, 0.02))

In [ ]:
# Predykcje dla każdego punktu siatki, aby narysować granicę decyzyjną
Z = perceptron.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.figure(figsize=(8, 6))
plt.contourf(xx, yy, Z, alpha=0.4, cmap=plt.cm.Paired)
plt.scatter(X0[:, 0], X0[:, 1], color='red', marker='o', label='Klasa 0 (trening)')
plt.scatter(X1[:, 0], X1[:, 1], color='blue', marker='x', label='Klasa 1 (trening)')
plt.scatter(new_points[:, 0], new_points[:, 1], color='green', marker='s', label='Punkty testowe')
plt.xlabel('Cecha 1')
plt.ylabel('Cecha 2')
plt.title('Granica decyzyjna perceptronu')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Zakładane etykiety dla punktów testowych (ground truth)
expected_labels = np.array([0, 1, 1, 0, 1])

# Obliczenie skuteczności predykcji
accuracy = np.mean(test_predictions == expected_labels)
print("Skuteczność predykcji punktów testowych: {:.2f}%".format(accuracy * 100))


## Wnioski
Z przeprowadzonego eksperymentu wynika, że pojedynczy perceptron uczony metodą bezgradientową skutecznie klasyfikuje punkty należące do dwóch różnych klas, pod warunkiem że są one liniowo separowalne. Proces uczenia zapewnia korektę wag do momentu osiągnięcia idealnej klasyfikacji zbioru uczącego.

Jednak metoda ta ma swoje ograniczenia – nie sprawdzi się w przypadku zbiorów, które nie są liniowo separowalne. Dodatkowo, szybkość uczenia może zależeć od wartości początkowych wag i współczynnika uczenia.
